# Polygonal Map Generation using Voronoi Maps

See Design.org for the research. Minimal description here as we develop the
idea.

First step is to recreate the [tutorial](http://docs.scipy.org/doc/scipy-0.14.0/reference/tutorial/spatial.html) on using SciPy's spatial data structures
and algorithms.


In [1]:
from scipy.spatial import Voronoi, voronoi_plot_2d, KDTree
import numpy as np
import matplotlib.pyplot as plt
import attr

%matplotlib inline
plt.rcParams['figure.figsize'] = [10.0, 6.0]

%config InlineBackend.figure_format = 'svg'

# Do some stuff

In [3]:
import tcod

ImportError: DLL load failed: The specified procedure could not be found.

In [4]:
points = np.array([[0., 0.], [0., 1.], [0., 2.], [1., 0.], [1., 1.], [1., 2.],
                   [2., 0.], [2., 1.], [2., 2.]])


In [5]:
plt.plot(points[:,0], points[:,1], 'ko')
plt.show()

<Figure size 432x288 with 1 Axes>

In [6]:
vor = Voronoi(points)
vor.vertices

array([[0.5, 0.5],
       [1.5, 0.5],
       [0.5, 1.5],
       [1.5, 1.5]])

In [7]:
voronoi_plot_2d(vor)

<Figure size 432x288 with 1 Axes>

<Figure size 432x288 with 1 Axes>

The values in `vor.regions` denote the set of points from `vor.vertices` forming
the Polygonal edges of the Voronoi regions. Negative numbers indicate points at
infinity.


In [ ]:
vor.regions

`vor.ridge_vertices` indicate lines in 2-D that separate the regions. Voronoi
ridges are perpendicular to lines drawn between the input points.
`vor.ridge_points` are the two points each ridge corresponds to.

In [ ]:
vor.ridge_points

In [ ]:
%pwd

In [ ]:
%run MapChunk.py

### Generating a Map

Grid dimensions corresponds roughly to number of points.

Grid width * grid height = Points/polygons.

In [ ]:
width, height = 100, 100
seed = np.random.random((width*height, 2))

In [ ]:
plt.plot(seed[:,0], seed[:,1], 'ko')

In [ ]:
relaxed = relax(seed, 4)

In [ ]:
polymap = Voronoi(relaxed)

Zoom in a bit to see if this is working.

In [ ]:
fig = plt.plot(relaxed[:,0], relaxed[:,1], 'ko')
plt.xlim([0.4, 0.5])
plt.ylim([0.4, 0.5])

In [ ]:
voronoi_plot_2d(polymap)
plt.xlim([0.4, 0.5])
plt.ylim([0.4, 0.5])


In [ ]:
nearest = KDTree(polymap.points)

In [ ]:
nearest.query([0.983, 0.148])

In [ ]:
nearest.data[0]

## Map Representation

Now comes the task of storing points and polygons to a graph.

 - Voronoi.vertices gives list of vertices (bound?) forming voronoi regions,

 - voronoi.ridge_vertices tells us how the vertices are connected.

 - voronoi.ridge_points are the two points on either side of a given ridge.

 - voronoi.region are the vertices that form a polygon for a given voronoi
   region.

   The relaxing algorithm will take a point and get the voronoi region for that
   point. If there are no negative indices then computing the centroid is easy.
   If not, then the question is where do(es) the infinite point(s) lay?

   The number of ridges for a region = number of edges for the polygon. Number
   of vertices for polygon == n_edges

   First do we find the ridge_points connected to point?

In [ ]:
polymap.ridge_vertices[2]

Graph is the fundamental data structure for representing the world.


In [ ]:
import networkx as nx


One graph for polygons, another for polygon corner (vertex?).

Nodes are the points/vertices, but their values need to be hashable so make the
nodes the indexes of the points/vertices in the Voronoi object.

Edges are enumerated in ridge_vertices and ridge_points. ridge_points shows
relationships between points, ridge_vertices between vertices.

In [ ]:
@attr.s
class PolyFeature(object):
    idx = attr.ib()
    parent = attr.ib()
    center = attr.ib()
    elevation = attr.ib(default=0.0)
    precipitation = attr.ib(default=0.0)
    temperature = attr.ib(default=0.0)
    color = attr.ib(default=0.0)

In [ ]:
corners = nx.Graph()
polys = nx.Graph()
points = relax(np.random.random((100*100,2)), 4)
polymap = Voronoi(points)
dist_map = KDTree(points)

In [ ]:
len(polymap.points)

In [ ]:
for p in range(len(polymap.points)):
    f = PolyFeature(idx=p, map=polys, center=polymap.points[p])
    polys.add_node(p, feature=f)

for rp in polymap.ridge_points:
    polys.add_edge(*rp.tolist())

for c in range(len(polymap.vertices)):
    corners.add_node(c)

for rv in polymap.ridge_vertices:
    corners.add_edge(*rv)

In [ ]:
polys.node[1]

In [ ]:
dist_map.query([0.0, 0.0])

In [ ]:
polys.node[9575]

In [ ]:
polymap.points[9575]

## Exploration

In [ ]:
%run MapChunk.py

In [ ]:
map = PolygonMap(feature_cnt=500, width=100.0, height=100.0)

In [ ]:
map.width

In [ ]:
%timeit map[0,0]

In [ ]:
hm = map.terrain_to_hm('elevation')

In [ ]:
plt.plot(map.points[:,0], map.points[:,1], 'ko')